In [1]:
import numpy as np

In [2]:
matrix = np.array([
  np.array([20, 7, 9], dtype=float),
  np.array([7, 30, 8], dtype=float),
  np.array([9, 8, 30], dtype=float),
])

In [3]:
b = np.array([
  np.array([16], dtype=float),
  np.array([38], dtype=float),
  np.array([38], dtype=float),
])

In [4]:
# Find the absolute maximum from M(p:n, p:n) to use it as a pivot
# Parameters:
#    M - Matrix
# Return
#   - The row and column index of the biggest value in the matrix
def find_biggest_pivot_value_id(M, p):
    pivot = 0
    row_idx = 0
    col_idx = 0
    n = M[0].size
    M_ = abs(M[p:n].T[p:n].T)
    if np.max(M_) > abs(np.min(M_)):
        pivot = np.max(M_)
        row_idx = np.amax(M_, axis=1).argmax()
        col_idx = np.amax(M_, axis=0).argmax()
    else:
        pivot = abs(np.min(M_))
        row_idx = np.amin(M_, axis=1).argmin()
        col_idx = np.amin(M_, axis=0).argmin()
                
    return pivot, row_idx + p, col_idx + p
find_biggest_pivot_value_id(matrix, 2)

(30.0, 2, 2)

In [5]:
# Swap Lines in matrix
def swap_lines(M, line1, line2):
    M_copy = M.copy()
    M_copy[[line1, line2]] = M_copy[[line2, line1]]
    return M_copy

In [6]:
# Swap columns in a matrix and result
def swap_columns(M, b, col1, col2):
    M_copy = M.copy()
    b_ = b.copy()    
    M_copy[:, [col1, col2]] = M_copy[:, [col2, col1]]
    b_[[col1, col2]] = b_[[col2, col1]]
    return M_copy, b_
# swap_columns(matrix, b, 0, 1)

In [34]:
def LU_with_total_pivot(Matrix, b):
    b_ = b.copy()
    lines_size = len(Matrix)
    col_size = Matrix[0].size
    L = np.identity(lines_size)
    U = Matrix.copy()
    
    # build the permutation vector
    perm = np.arange(col_size);
    
    for c in range(col_size-1): # columns
        # we find the absolute maximum from A(c:n, c:n) to use it as a pivot
        
        if (abs(U[c][c]) < 0.0001):
            pivot, idx_biggest_line, idx_biggest_col = find_biggest_pivot_value_id(U, c)     
            if (idx_biggest_line != c and idx_biggest_col > c-1):
                U = swap_lines(U, idx_biggest_line, c) 
            if (idx_biggest_col != c and idx_biggest_col > c-1):
                U, perm = swap_columns(U, perm, idx_biggest_col, c)       
        
        for l in range(c+1, lines_size): # lines
            
            alfa = - U[l][c] / U[c][c]             
            L[l][c] = - alfa         
            U[l] += alfa*U[c]
            U[l][c] = 0
            b_[l][0] += alfa*b_[c][0]
        
    return L, U, b_, perm
L, U, b_, perm = LU_with_total_pivot(matrix, b)

print(L)
print(U)
print(b_)
print(perm)

[[1.         0.         0.        ]
 [0.35       1.         0.        ]
 [0.45       0.17604356 1.        ]]
[[20.          7.          9.        ]
 [ 0.         27.55        4.85      ]
 [ 0.          0.         25.09618875]]
[[16.        ]
 [32.4       ]
 [25.09618875]]
[0 1 2]


In [35]:
b_

array([[16.        ],
       [32.4       ],
       [25.09618875]])

## Ly = b

In [37]:
n = b_.size
y = np.zeros(n)

for k in range(n):
    s = 0
    for j in range (k+1, n):
        s += L[k][j]*y[j]
    y[k] = (b_[k] - s) / L[k][k]
y

array([16.        , 32.4       , 25.09618875])

## Ux = y

In [40]:
last_index = n-1
x = np.zeros(n)
x[last_index] = y[last_index]/U[last_index][last_index]
for k in range(last_index-1, 0, -1 ):
    s = 0
    for j in range(k+1, n):
        s += U[k][j]*x[j]
    x[k] = (y[k] - s) / U[k][k]
x

array([0., 1., 1.])